In [1]:
import requests
import pandas as pd
from API_Key import api_key

pages = list(range(1,201))

popular = pd.DataFrame()

for page in pages:
    movieData = pd.DataFrame(requests.get(f'https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=en-US&page={page}').json()['results'])
    popular = popular.append(movieData)


popular = popular[popular['adult'] == False]
popular

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/lNyLSOKMMeUPr1RsL4KcRuIXwHt.jpg,"[878, 28, 12]",580489,en,Venom: Let There Be Carnage,After finding a host body in investigative rep...,13071.277,/rjkmN1dniUHVYAtwuV3Tji7FsDO.jpg,2021-09-30,Venom: Let There Be Carnage,False,7.2,3726
1,False,/dK12GIdhGP6NPGFssK2Fh265jyr.jpg,"[28, 35, 80, 53]",512195,en,Red Notice,An Interpol-issued Red Notice is a global aler...,4264.425,/wdE6ewaKZHr62bLqCn7A2DiGShm.jpg,2021-11-04,Red Notice,False,6.8,1623
2,False,/cinER0ESG0eJ49kXlExM0MEWGxW.jpg,"[28, 12, 14]",566525,en,Shang-Chi and the Legend of the Ten Rings,Shang-Chi must confront the past he thought he...,3517.554,/1BIoJGKbXjdFDAqUEiA2VHqkK1Z.jpg,2021-09-01,Shang-Chi and the Legend of the Ten Rings,False,7.9,3848
3,False,/vIPIyTJqcgOKgKcExCvTDpLpTYW.jpg,"[28, 12, 878, 14]",634649,en,Spider-Man: No Way Home,Peter Parker is unmasked and no longer able to...,3317.946,/1g0dhYtq4irTY1GPXvft6k4YLjm.jpg,2021-12-15,Spider-Man: No Way Home,False,0.0,0
4,False,/xGrTm3J0FTafmuQ85vF7ZCw94x6.jpg,"[18, 36, 12]",589761,ru,Чернобыль,The aftermath of a shocking explosion at the C...,3245.023,/kfQJQWFEoWRVBH8FUKnT0HX1yRS.jpg,2021-04-15,Chernobyl: Abyss,False,6.2,215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,False,/p01c4KaaI8oqmKQqfrQiqYlHR6W.jpg,"[28, 53, 9648]",146198,en,Triple 9,A gang of criminals and corrupt cops plan the ...,18.418,/hMW4p1CBF8Q94PqQk8b2EyT4IJk.jpg,2016-02-19,Triple 9,False,5.9,1710
16,False,/76sCO4mSEcsWyOxRCvDOAhL2DnO.jpg,"[80, 9648, 53, 28]",693113,en,Midnight in the Switchgrass,FBI Agent Karl Helter and his partner Rebecca ...,18.407,/1OTSuh2HNcTRfme8caU8DjgJ39K.jpg,2021-07-23,Midnight in the Switchgrass,False,6.5,178
17,False,/nb8pMW3o85jATvhz7y6clAFlL1u.jpg,"[18, 80]",10474,en,The Basketball Diaries,A high school basketball player’s life turns u...,18.401,/AhvO1GGDPIgN0hOqZEgaFCbswMK.jpg,1995-04-21,The Basketball Diaries,False,7.4,1424
18,False,/cO8zcdPhLT3UdykkXCvfhIVrgHK.jpg,"[28, 12, 878]",9355,en,Mad Max Beyond Thunderdome,Mad Max becomes a pawn in a decadent oasis of ...,18.401,/jJlxcEVVUHnrUeEkQ0077VeHQpb.jpg,1985-06-29,Mad Max Beyond Thunderdome,False,6.2,2084


In [2]:
popular_features = popular[['id','title','genre_ids','release_date','popularity','vote_average','vote_count']]
popular_features = popular_features.rename(columns={
                                        "id":"movie_id",
                                        "popularity":"movie_popularity"
                                        })

In [3]:
id_list = popular_features['movie_id'].tolist()
revenue_list = []
budget_list = []

for movie in id_list:
    x = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['revenue']
    y = requests.get(f'https://api.themoviedb.org/3/movie/{movie}?api_key={api_key}&language=en-US').json()['budget']
    revenue_list.append(x)
    budget_list.append(y)

revenue_df = pd.DataFrame(
    {'movie_id':id_list,
    'budget': budget_list,
    'revenue':revenue_list
    })

revenue_df.reset_index(drop = True)

,movie_id,budget,revenue
0,580489,110000000,454000000
1,512195,200000000,0
2,566525,150000000,430238384
3,634649,0,0
4,589761,9100000,5370393
...,...,...,...
3995,146198,20000000,12639297
3996,693113,15000000,0
3997,10474,2000000,2381087
3998,9355,10000000,36230219


In [4]:

Movie_Data = popular_features.merge(revenue_df, on='movie_id', how='inner')
Movie_Data.count()

movie_id            4000
title               4000
genre_ids           4000
release_date        3991
movie_popularity    4000
vote_average        4000
vote_count          4000
budget              4000
revenue             4000
dtype: int64

In [5]:
ratings = []
x = 0
for movie_id in Movie_Data['movie_id']:
    movie_rating = requests.get(f'http://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=release_dates').json()['release_dates']['results']
    for rating in movie_rating:
        if rating['iso_3166_1'] == 'US':
            try:
                ratings.append(rating['release_dates'][1]["certification"])
                x=1
            except IndexError:
                x=1
                ratings.append(rating['release_dates'][0]["certification"])
    if x == 0:
        ratings.append('')
    x=0

In [6]:
Movie_Data['rating'] = ratings
Movie_Data

,movie_id,title,genre_ids,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating
0,580489,Venom: Let There Be Carnage,"[878, 28, 12]",2021-09-30,13071.277,7.2,3726,110000000,454000000,PG-13
1,512195,Red Notice,"[28, 35, 80, 53]",2021-11-04,4264.425,6.8,1623,200000000,0,PG-13
2,566525,Shang-Chi and the Legend of the Ten Rings,"[28, 12, 14]",2021-09-01,3517.554,7.9,3848,150000000,430238384,PG-13
3,634649,Spider-Man: No Way Home,"[28, 12, 878, 14]",2021-12-15,3317.946,0.0,0,0,0,PG-13
4,589761,Chernobyl: Abyss,"[18, 36, 12]",2021-04-15,3245.023,6.2,215,9100000,5370393,
...,...,...,...,...,...,...,...,...,...,...
3995,146198,Triple 9,"[28, 53, 9648]",2016-02-19,18.418,5.9,1710,20000000,12639297,R
3996,693113,Midnight in the Switchgrass,"[80, 9648, 53, 28]",2021-07-23,18.407,6.5,178,15000000,0,R
3997,10474,The Basketball Diaries,"[18, 80]",1995-04-21,18.401,7.4,1424,2000000,2381087,R
3998,9355,Mad Max Beyond Thunderdome,"[28, 12, 878]",1985-06-29,18.401,6.2,2084,10000000,36230219,PG-13


In [7]:
filtered_table = Movie_Data[Movie_Data['budget']!= 0]
filtered_table = filtered_table[filtered_table['revenue'] != 0]
filtered_movies = filtered_table[filtered_table['rating'] != '']
filtered_movies

,movie_id,title,genre_ids,release_date,movie_popularity,vote_average,vote_count,budget,revenue,rating
0,580489,Venom: Let There Be Carnage,"[878, 28, 12]",2021-09-30,13071.277,7.2,3726,110000000,454000000,PG-13
2,566525,Shang-Chi and the Legend of the Ten Rings,"[28, 12, 14]",2021-09-01,3517.554,7.9,3848,150000000,430238384,PG-13
8,370172,No Time to Die,"[12, 28, 53]",2021-09-29,1938.906,7.6,2283,242000000,758000000,PG-13
10,524434,Eternals,"[28, 12, 878, 18]",2021-11-03,1339.598,7.1,1293,200000000,368000000,PG-13
13,550988,Free Guy,"[35, 28, 12, 878]",2021-08-11,1246.828,7.8,3909,110000000,331096766,PG-13
...,...,...,...,...,...,...,...,...,...,...
3993,11260,Meet Dave,"[35, 878, 12, 10751]",2008-07-08,18.426,5.3,1092,60000000,50650079,PG
3995,146198,Triple 9,"[28, 53, 9648]",2016-02-19,18.418,5.9,1710,20000000,12639297,R
3997,10474,The Basketball Diaries,"[18, 80]",1995-04-21,18.401,7.4,1424,2000000,2381087,R
3998,9355,Mad Max Beyond Thunderdome,"[28, 12, 878]",1985-06-29,18.401,6.2,2084,10000000,36230219,PG-13


In [8]:
filtered_movies.count()

movie_id            1905
title               1905
genre_ids           1905
release_date        1905
movie_popularity    1905
vote_average        1905
vote_count          1905
budget              1905
revenue             1905
rating              1905
dtype: int64

In [9]:

titleNid = filtered_movies[['movie_id','title']]

titleNid = titleNid.reset_index(drop = True)

titleNid.to_csv('../Tables/titleNid.csv',index = False)

filtered_movies.to_csv('../Tables/Movie_Data.csv', index = False)

In [10]:
genre = requests.get(f'https://api.themoviedb.org/3/genre/movie/list?api_key={api_key}&language=en-US').json()['genres']
genre_df = pd.DataFrame(genre)
genre_df
genre_df.to_csv('../Tables/Genres.csv',index = False)